In [1]:
import os
import subprocess
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock

# ---------- Configuration ----------
MAX_WORKERS = 1
EXCLUDE_PREFIXES = [
    '__pycache__', 
    '.ipynb_checkpoints', 
    'base', 
    'completed runs', 
    'to do', 
    'analysis'
]

# Lock for thread-safe printing
print_lock = Lock()

# ---------- Core Helpers ----------
def run_notebook(folder: str, notebook_name: str) -> str:
    """
    Execute a Jupyter notebook in place using nbconvert.
    Returns a status string with execution time.
    All printing inside this function is thread-safe.
    """
    notebook_path = os.path.join(folder, notebook_name)
    if not os.path.exists(notebook_path):
        with print_lock:
            print(f"SKIP: {notebook_name} not found in {folder}", flush=True)
        return f"SKIP: {notebook_name} not found in {folder}"

    with print_lock:
        print(f"🔹 Running {notebook_path}...", flush=True)

    start_time = time.time()
    try:
        subprocess.run(
            [
                'jupyter', 'nbconvert', '--to', 'notebook',
                '--execute', notebook_path, '--inplace',
                '--ExecutePreprocessor.timeout=-1'
            ],
            check=True,
            capture_output=True,
            text=True
        )
        elapsed = time.time() - start_time
        return f"✅ SUCCESS: {folder}/{notebook_name} (took {elapsed:.2f}s)"
    except subprocess.CalledProcessError as e:
        elapsed = time.time() - start_time
        stderr_clean = e.stderr.strip()
        return f"❌ ERROR: {folder}/{notebook_name} - {stderr_clean} (took {elapsed:.2f}s)"

def find_folders(base_dir: str = '.') -> list[str]:
    """Return a list of folders to process, excluding certain prefixes."""
    return [
        d for d in os.listdir(base_dir)
        if os.path.isdir(d) and not any(d.startswith(prefix) for prefix in EXCLUDE_PREFIXES)
    ]

def run_in_parallel(folders: list[str], notebook_names: list[str]):
    """Run one or more notebooks across all folders in parallel, with thread-safe printing."""
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [
            executor.submit(run_notebook, folder, nb)
            for folder in folders
            for nb in notebook_names
        ]

        for future in as_completed(futures):
            msg = future.result().strip()
            with print_lock:
                print(msg)
    with print_lock:
        print("✅ All done!")

# ---------- Specialized Variants ----------
def run_random_perturbs_parallel():
    """Run Random Perturbs.ipynb across all model_* folders."""
    folders = find_folders()
    with print_lock:
        print(f"Found {len(folders)} folders for Random Perturbs.")
    run_in_parallel(folders, ["Random Perturbs.ipynb"])

def run_volume_parallel():
    """
    Run all notebooks containing specific keywords (Volume Estimation, Volume Cutoff, Test Accuracy)
    across all folders. Easy to comment out unwanted notebook types by editing valid_names list.
    """
    folders = find_folders()
    with print_lock:
        print(f"Found {len(folders)} folders for Volume/Test Accuracy notebooks.")

    # ✅ Editable list of allowed notebook name substrings
    valid_names = [
        "Volume Estimation",
        "Volume Cutoff",
        "Test Accuracy"
    ]

    notebook_names = []
    if folders:
        reference_folder = folders[0]
        for nb in os.listdir(reference_folder):
            if nb.endswith(".ipynb") and any(name in nb for name in valid_names):
                notebook_names.append(nb)

    if not notebook_names:
        with print_lock:
            print("⚠️ No matching notebooks found in reference folder.")
        return

    with print_lock:
        print(f"Will run these notebooks: {notebook_names}")
    run_in_parallel(folders, notebook_names)


def run_nb_parallel():
    """Generic function for ad-hoc notebook runs (prompt user)."""
    notebook_name = input("Enter notebook name (without .ipynb): ").strip() + ".ipynb"
    folders = find_folders()
    with print_lock:
        print(f"Found {len(folders)} folders to process.")
    run_in_parallel(folders, [notebook_name])

run_volume_parallel()

Found 10 folders for Volume/Test Accuracy notebooks.
Will run these notebooks: ['Test Accuracy.ipynb', 'Volume Cutoff.ipynb', 'Volume Estimation Pipeline.ipynb']
🔹 Running model_0_data_10\Test Accuracy.ipynb...
✅ SUCCESS: model_0_data_10/Test Accuracy.ipynb (took 13.94s)
🔹 Running model_0_data_10\Volume Cutoff.ipynb...
✅ SUCCESS: model_0_data_10/Volume Cutoff.ipynb (took 8.06s)
🔹 Running model_0_data_10\Volume Estimation Pipeline.ipynb...
✅ SUCCESS: model_0_data_10/Volume Estimation Pipeline.ipynb (took 18.89s)
🔹 Running model_1_data_11\Test Accuracy.ipynb...
✅ SUCCESS: model_1_data_11/Test Accuracy.ipynb (took 7.10s)
🔹 Running model_1_data_11\Volume Cutoff.ipynb...
✅ SUCCESS: model_1_data_11/Volume Cutoff.ipynb (took 7.89s)
🔹 Running model_1_data_11\Volume Estimation Pipeline.ipynb...
✅ SUCCESS: model_1_data_11/Volume Estimation Pipeline.ipynb (took 19.53s)
🔹 Running model_2_data_12\Test Accuracy.ipynb...
🔹 Running model_2_data_12\Volume Cutoff.ipynb...
✅ SUCCESS: model_2_data_12/Test

✅ SUCCESS: model_0_data_10/Test Accuracy.ipynb (took 7.16s)
🔹 Running model_0_data_10\Volume Cutoff.ipynb...


✅ SUCCESS: model_0_data_10/Volume Cutoff.ipynb (took 8.04s)
🔹 Running model_0_data_10\Volume Estimation Pipeline.ipynb...


✅ SUCCESS: model_0_data_10/Volume Estimation Pipeline.ipynb (took 18.43s)
🔹 Running model_1_data_11\Test Accuracy.ipynb...


✅ SUCCESS: model_1_data_11/Test Accuracy.ipynb (took 7.04s)
🔹 Running model_1_data_11\Volume Cutoff.ipynb...


...

✅ SUCCESS: model_8_data_18/Volume Cutoff.ipynb (took 8.10s)
🔹 Running model_8_data_18\Volume Estimation Pipeline.ipynb...


✅ SUCCESS: model_8_data_18/Volume Estimation Pipeline.ipynb (took 18.26s)
🔹 Running model_9_data_19\Test Accuracy.ipynb...


✅ SUCCESS: model_9_data_19/Test Accuracy.ipynb (took 6.98s)
🔹 Running model_9_data_19\Volume Cutoff.ipynb...


✅ SUCCESS: model_9_data_19/Volume Cutoff.ipynb (took 8.11s)
🔹 Running model_9_data_19\Volume Estimation Pipeline.ipynb...


✅ SUCCESS: model_9_data_19/Volume Estimation Pipeline.ipynb (took 18.73s)
✅ All done!
